In [97]:
import numpy as np
import pickle
import scipy.stats as st
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel

In [129]:
XX = np.load(r'D:\Network Security\KL Control\robotarium_python_simulator\rps\State_Data_Nominal.npy',allow_pickle=True)
UU = np.load(r'D:\Network Security\KL Control\robotarium_python_simulator\rps\Input_Dataset_new.npy',allow_pickle=True)

In [130]:
X = []
X_plot = []

X_Prev = []
X_plot_Prev = []

U = []
U_plot = []

for i in range(len(XX)):
    X.append(np.array(XX[i]))
    X_plot.append(np.array(XX[i]))

X = np.concatenate(X, axis=0)
X = np.reshape(X, (-1, 2))

U = []
for i in range(len(UU)):
    U.append(np.array(UU[i]))
    U_plot.append(np.array(UU[i]))

U = np.concatenate(U, axis=0)
U = np.reshape(U, (-1, 2))

In [131]:
X1 = XX[0,:,:,:]
X2 = XX[:2,:,:,:].reshape((-1,2))
X3 = XX[:,:,:,:].reshape((-1,2))

U1 = UU[0,:,:,:]
U2 = UU[:2,:,:,:].reshape((-1,2))
U3 = UU[:,:,:,:].reshape((-1,2))

In [123]:
def nominal_model_step(x, velocities, time_step):
    poses = np.zeros((2, 1))
    poses[0] = 1.0 * x[0] + time_step * velocities[0] + 0.0 * x[0]
    poses[1] = 1.0 * x[1] + time_step * velocities[1] + 0.0* x[1]
    return poses

In [125]:
X_train = np.hstack((XX[2][:-1].reshape(-1,2),UU[2].reshape(-1,2))).reshape((-1,4))
Y_train = XX[2,1:].reshape((-1,2))

# Define the GP model with RBF kernel
# Define the kernel for the Gaussian Process
kernel = ConstantKernel() * RBF(length_scale=[0.5,0.5,0.5,0.5]) + WhiteKernel()
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

# Train the GP model
gp.fit(X_train, Y_train)

IndexError: index 2 is out of bounds for axis 0 with size 1

In [124]:
X_train = np.hstack((X2[:-1,:],U2[:,:])).reshape((-1,4))
Y_train = X2[1:,:].reshape((-1,2))

# Define the GP model with RBF kernel
# Define the kernel for the Gaussian Process
kernel = ConstantKernel() * RBF(length_scale=[0.5,0.5,0.5,0.5]) + WhiteKernel()
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

# Train the GP model
gp.fit(X_train, Y_train)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 712 and the array at index 1 has size 1000

In [118]:
# Example of predicting with the GP model
time_step = 0.0333
test_state = np.array([0.5, -0.5])
test_velocities = np.array([-0.15, 0.15])
test_input = np.hstack((test_state, test_velocities)).reshape(1, -1)
predicted_next_state,covn = gp.predict(test_input,return_cov=True)

next_state_test = nominal_model_step(test_state, test_velocities, time_step)

print("Predicted next state:", predicted_next_state.T)
print('actual next state:',next_state_test)
print('Covariance:',covn)

Predicted next state: [[ 0.49882429]
 [-0.48433805]]
actual next state: [[ 0.495005]
 [-0.495005]]
Covariance: [[[0.00013345 0.00013345]]]


In [10]:
gp.kernel_.get_params()

{'k1': 9.8**2 * RBF(length_scale=[20.6, 19.4, 401, 417]),
 'k2': WhiteKernel(noise_level=1e-05),
 'k1__k1': 9.8**2,
 'k1__k2': RBF(length_scale=[20.6, 19.4, 401, 417]),
 'k1__k1__constant_value': 96.10841278607474,
 'k1__k1__constant_value_bounds': (1e-05, 100000.0),
 'k1__k2__length_scale': array([ 20.61778115,  19.42858169, 401.04667891, 416.52370577]),
 'k1__k2__length_scale_bounds': (1e-05, 100000.0),
 'k2__noise_level': 9.999999999999997e-06,
 'k2__noise_level_bounds': (1e-05, 100000.0)}

In [113]:
with open("GP_nominal_2.dump","wb") as f:
    pickle.dump(gp, f)